# Convert ERA5 variable units to the values expected by NEMO

Variables read in by NEMO: sn_wndi (uas), sn_wndj (vas), sn_qsr (rsds), sn_qlw (rlds), sn_tair (tas), sn_humi (huss), sn_prec (tprecip), sn_snow (prsn), sn_slp (psl).

Variable conversions needed:
- U wind ERA5 --> eastward near surface wind (which then is rotated to the NEMO grid U and V online)
- V wind ERA5 --> northward near surface wind
- Humidity: convert 2 m dewpoint temperature --> near surface specific humidity

Unit conversions needed:
- snowfall flux: m of water equivalent --> kg /m2 /s
- total solid + liquid precip: m --> kg /m2 /s
- surface downwelling LW radiation: J/m2 --> W/m2
- surface downwelling SW radiation: J/m2 --> W/m2

In [2]:
import numpy as np
import xarray as xr
import glob

#### Functions

In [9]:
def convert_wind(file_U='era5_u10_1979_daily_averages.nc', variableU='u10',
                 file_V='era5_v10_1979_daily_averages.nc', variableV='v10',
                 dataset='ERA5', folder='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/'):
    # ERA5 gives wind in U and V grid direction instead of eastward and northward
    # https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398
    
    Uwind = xr.open_dataset(f'{folder}{file_U}')[variableU]
    Vwind = xr.open_dataset(f'{folder}{file_V}')[variableV]

    speed = np.sqrt(Uwind**2 + Vwind**2)

    theta = 90 # eastward or 0 for northerly
    u_dir = - speed * np.sin(theta)
    v_dir = - speed * np.cos(theta)
                     
    return u_dir, v_dir

def convert_radiation(file_rad='era5_strd_1979_daily_averages.nc', variable='strd', 
                      dataset='ERA5', folder='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/'):
    if dataset=='ERA5':
        # ERA5 is in J m-2, convert to Watt m-2 = J m-2 s-1, so divide by the accumulation period in seconds
        # In this case, the files are daily averages of the original hourly files. So, the J/m-2 is actually the accumulation over an hour. 
        QW = xr.open_dataset(f'{folder}{file_rad}')[variable] # shortwave or longwave radiation
        
        return QW/3600
    else:
        raise Exception('Only currently set up to convert ERA5 units to nemo units')

def convert_precip(file_precip='era5_tp_1979_daily_averages.nc', variable='tp', 
                   dataset='ERA5', folder='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/'):
    if dataset=='ERA5':
        # ERA5 is in m of water equivalent, convert to kg m-2 s-1, so need to divide by the accumulation period, and convert density
        precip = xr.open_dataset(f'{folder}{file_precip}')[variable]
        # m --> m/s --> kg/m2/s
        rho_water = 1000 # kg/m3
        precip = (precip / 3600) * rho_water # total precip is in meters of water equivalent
        
        return precip
        
    else:
        raise Exception('Only currently set up to convert ERA5 units to nemo units')

def convert_humidity(file_dew='era5_d2m_1979_daily_averages.nc', variable_dew='d2m',
                     file_slp='era5_msl_1979_daily_averages.nc', variable_slp='msl',
                     dataset='ERA5', folder='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/'):
    if dataset=='ERA5':
        # ERA5 does not provide specific humidity, but gives the 2 m dewpoint temperature in K
        # Conversion assumes temperature is in K and pressure in Pa.
        # Based off: https://confluence.ecmwf.int/pages/viewpage.action?pageId=171411214

        dewpoint = xr.open_dataset(f'{folder}{file_dew}')[variable_dew]
        surface_pressure = xr.open_dataset(f'{folder}{file_slp}')[variable_slp]
        
        # constants: # note that it could be somewhat different over ice?
        a1 = 611.21; a3 = 17.502; a4=32.19; T0=273.16;
        Rdry = 287.0597; Rvap=461.5250; 
        # calculation:
        vapor_pressure = a1*np.exp(a3*(dewpoint.values - T0)/(dewpoint.values - a4)) # E saturation water vapour from Teten's formula
        spec_humidity  = (Rdry / Rvap) * vapor_pressure / (surface_pressure - ((1-Rdry/Rvap)*vapor_pressure)) # saturation specific humidity
        
        return spec_humidity
    else:
        raise Exception('Only currently set up to convert ERA5 units to nemo units')
    

In [10]:
QLW = convert_radiation(file_rad='era5_strd_1979_daily_averages.nc', variable='strd', dataset='ERA5')
QSW = convert_radiation(file_rad='era5_ssrd_1979_daily_averages.nc', variable='ssrd', dataset='ERA5')
humidity = convert_humidity(file_dew='era5_d2m_1979_daily_averages.nc', variable_dew='d2m', 
                            file_slp='era5_msl_1979_daily_averages.nc', variable_slp='msl', dataset='ERA5')
tprecip  = convert_precip(file_precip='era5_tp_1979_daily_averages.nc', variable='tp', dataset='ERA5')
snow_flx = convert_precip(file_precip='era5_sf_1979_daily_averages.nc', variable='sf', dataset='ERA5')